In [77]:
from collections import defaultdict
from helper import get_softmax
import tensorflow as tf
import numpy as np
import random

global train_lib

def get_importance(state=None):
    return np.array([0.25, 0.25, 0.5])

def get_combined_probs(state):
    importance = get_importance(state)
    summed_probs = np.array([0., 0., 0.])
    for i in range(len(train_lib)):
        probs = get_softmax(train_lib[i], tau=1.0)
        summed_probs += importance[i] * probs
    return summed_probs
oh, wie wars
def get_action(state=None, tau=1.0):
    """ Returns an action selected through softmax. """
    # print(self.step_current, self.tau, self.batch_size)
    return np.random.choice(3, p=get_combined_probs(state))

In [82]:
train_lib = (np.array([[1, 1, 1]]),
             np.array([[0, 1, 2]]),
             np.array([[0, 0, 1]]))

action = get_action()
print("Action:", action)
# probabilities for each action
#probs = defaultdict()
#weighted_probs = defaultdict()

#print("summed_probs = " + str(summed_probs) + " --> sum = " + str(sum(summed_probs)))


('Action:', 2)
